# Assignment 7: Decision Trees

_**NOTE**: this assignment is only mandatory for students registered for CSC 8515; students registered for CSC 4515 are **not** required to complete this assignment._

For this assignment, your goal is to implement the ID3 algorithm (greedy information-gain based decision tree induction).  You should use the Congressional Voting Records data set, the same one that we used with Naive Bayes, and for the same reason; it is a discrete multivariate dataset, which is well suited to decision trees.  

Note that while scikit-learn does have decision trees, it doesn't really handle discrete data well, so the library version may not produce exactly the same trees that your algorithm will.

Implement and test your algorithm; be sure you can print the actual tree (a simple text-based visualization is fine), as well as calculating classification accuracy on held-out testing data.

**CSC 8515 - Machine Learning  
Assignment 7  
Scaffolding by Dr. Ben Mitchell  
Assignment completed by: James Fung  **

Some resources used:
https://stackoverflow.com/questions/10148818/numpy-how-to-iterate-over-columns-of-array
https://stackoverflow.com/questions/47448931/why-cant-i-iterate-through-nested-keys
https://stackoverflow.com/questions/40434121/recursive-programming-in-decision-tree
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
http://scikit-learn.org/stable/modules/tree.html

In [1]:
import pandas as pd
import numpy as np
import scipy

Section: **Data manipulation.**

In [2]:
#Column names.
colNames = ['party']
for i in range(16):
    colNames.append('Bill '+str(i))

#Read the data.
voting = pd.read_csv('house-votes-84.data', header=None, names=colNames )

#Shuffle data on 70/30.
shuffled = voting.sample(frac=1)
trainFrac = 0.7
trainCount = int(len(voting) * trainFrac)
train = shuffled[:trainCount]
test = shuffled[trainCount:]

#Arrays with labels will be used for my implementation.
trainnumpy = train.values
testnumpy = test.values

#Arrays that are split apart are used for sklearns.
trainLabels = train['party'].values #Pull out just the column party without the row indexes. 
testLabels = test['party'].values
trainFeatures = train.drop(['party'], axis=1).values #Drop column as opposed to row, default is axis = 0.
testFeatures = test.drop(['party'], axis=1).values

# Section: My own implementation.

In [3]:
#Function to count number of times a class appears.
def uniquecounts(data):
    results={}
    for row in data:
        r=row[len(row)-1]
        if r not in results: results[r]=0
        results[r]+=1
    return results

In [4]:
#Entropy.
def entropy(data):
    import math
    counts=uniquecounts(data)
    ent=0.0
    for classes in counts.keys():
        p=float(counts[classes])/len(data)
        ent=ent-p*math.log2(p)
    return ent

In [5]:
#Create a function that will subset the dataset. This will be used for placing data into each child node. 

def subset(data,column,value):
    #Create an empty numpy array.
    newdata = np.empty((0,data.shape[1]))
    #Skip the labels.
    row = 1
    while row < data.shape[0]:
        #Subset on value.
        if data[row][column] == value:
            newdata = np.vstack((newdata,data[row]))
            row+=1
        #Skip row if value is not met.
        else:
            row+=1
    return newdata

In [6]:
#Create a function that will locate the best feature to divide the set.

def bestfeature(data):
    #Entropy of the dataset.
    baseentropy = entropy(data)
    #Initialize best information gain and best feature.
    bestig = float('inf')
    bestfeature = 0
    #Skip the class labels.
    col = 1
    while col < data.shape[1]:
        #Skip column if entirely pure.
        if entropy(data[:,col]) == 0:
            col += 1
        #Calculate information gain.
        elif baseentropy-entropy(data[:,col]) < bestig:
            bestig = baseentropy-entropy(data[:,col])
            bestfeature = col
        col += 1
    return bestfeature

In [7]:
def decisiontree(data,tree=None):
    
    #Find best feature.
    node = bestfeature(data)
    
    #Dictionary for tree - I don't think this is optimal but works for now.
    if tree is None:
        tree = {}
        tree[node] = {}
    
    #Used for splitting the data into subsets.
    features = np.unique(data[:,node])
    
    #Split the data.
    for feature in features:
        childdata = subset(data,node,feature)
        #Count how many times each label appears in each child node.
        label,count = np.unique(childdata[:,0],return_counts=True)
        
        #Check for purity.
        if len(count) == 1:
            tree[node][feature] = label[0]
        #Recurse.
        else:
            tree[node][feature] = decisiontree(childdata)
    return tree

In [8]:
#Train the tree on the training set, print the tree.
tree = decisiontree(trainnumpy)
tree

{2: {'?': {1: {'?': {6: {'?': {9: {'?': 'republican', 'y': {0: {}}}},
      'n': 'democrat',
      'y': {3: {'?': {0: {}}, 'n': 'democrat'}}}},
    'n': {3: {'?': 'republican',
      'n': {16: {'?': {5: {'n': {0: {}}, 'y': 'republican'}},
        'n': 'republican',
        'y': 'republican'}},
      'y': {5: {'?': 'democrat',
        'n': 'democrat',
        'y': {11: {'n': {0: {}}, 'y': 'democrat'}}}}}},
    'y': {13: {'?': 'republican',
      'n': 'democrat',
      'y': {3: {'n': 'republican', 'y': 'democrat'}}}}}},
  'n': {16: {'?': {6: {'?': 'democrat',
      'n': 'democrat',
      'y': {9: {'?': 'democrat', 'n': 'republican', 'y': 'democrat'}}}},
    'n': {11: {'?': {3: {'n': {0: {}}, 'y': 'democrat'}},
      'n': {10: {'n': {13: {'n': 'republican', 'y': 'republican'}},
        'y': 'republican'}},
      'y': 'democrat'}},
    'y': {15: {'?': {5: {'n': 'democrat',
        'y': {3: {'n': 'republican', 'y': {0: {}}}}}},
      'n': {8: {'?': 'republican',
        'n': {11: {'?': 'rep

In [9]:
#Predict new cases.
def predict(test,tree):
    for nodes in tree.keys():        
        value = test[nodes]
        tree = tree[nodes][value]
        #If it has not yet hit the bottom node, recurse. This is done by checking to see if the child node is a dict.
        if type(tree) is dict:
            prediction = predict(test, tree)
        #Return class label and break.
        else:
            prediction = tree
            break                       
    return prediction

#Test example.
predict(testnumpy[0],tree)

'democrat'

In [10]:
#Test the tree on the test set, calculate accuracy.

counter = 0
for i in range(0,len(testnumpy)-1):
    #For some reason, my code broke on a few examples where it was not able to find a resulting subkey, so on those
    #occurances I randomly selected a class label. Seems to occur on "?" for random columns.
    
    #If the prediction matches the label column. Note that the label column is not used in the algo as it is skipped
    #in the above functions.
    try:
        if predict(testnumpy[i],tree) == testnumpy[i][0]: counter += 1
    except:
    #Randomly choose a class as the prediction from above comment.
        import random
        classes = ['democrat','republican']
        pred = random.choice(classes)
        if pred == testnumpy[i][0]: counter += 1
accuracy = counter/len(testnumpy)
accuracy

0.8702290076335878

# Section: scikit learn for comparison

In [11]:
#Create a sklearns version to compare mine.

#Convert all values into a numeric using sklearns Label Encoder, as sklearns is not able to handle strings.
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

for i in range (0,trainFeatures.shape[1]):
    trainFeatures[:,i] = labelencoder.fit_transform(trainFeatures[:,i])
    
for i in range (0,testFeatures.shape[1]):
    testFeatures[:,i] = labelencoder.fit_transform(testFeatures[:,i])

#Train the tree on entropy/ID3.

from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf = clf.fit(trainFeatures,trainLabels)
skpredict = clf.predict(testFeatures)

#Calculate accuracy. 
skaccuracy = 0
for i,j in zip(skpredict,testLabels):
    if i==j: skaccuracy += 1
skaccuracy/len(testLabels)

0.9312977099236641